In [97]:
from itertools import product
from functools import reduce

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [107]:
n_units = 3
data = np.random.binomial(n=1, p=[1, 0, 1], size=(100, n_units))
alpha = .25

biases = np.random.randn(n_units)
weights = np.random.randn(n_units, n_units)

var_combinations = list(combinations(range(n_units), 2))

In [108]:
class Model:
    
    def __init__(self, weights, biases):
        self.weights = weights
        self.biases = biases
        
    def H(self, x):
        h = 0
        for i, j in var_combinations:
            h += self.weights[i, j] * x[i] * x[j]
        h += self.biases @ x
        return h
    
    def _unnormalized_likelihood(self, x):
        return np.exp(self.H(x))
    
    def marginal_likelihood(self, x):
        unnormalized_lik = 0
        for config in product(*[[0, 1] if el == ... else [el] for el in x]):
            config = np.array(config)
            unnormalized_lik += np.exp(self.H(config))
        return unnormalized_lik
    
    def likelihood(self, x, log=False):
        """
        Must have the dimensionality of the data observations. To marginalize, put ellipses (...)
        in the elements over which you wish to marginalize.
        """
        x = np.array(x)
        if not n_units in x.shape and len(x.shape) in (1, 2):
            raise('Please pass 1 or more points of `n_units` dimensions')
           
        # compute unnormalized likelihoods
        multiple_samples = len(x.shape) == 2
        if multiple_samples:
            likelihood = [self._unnormalized_likelihood(point) for point in x]
        else:
            likelihood = [self._unnormalized_likelihood(x)]
        
        # compute partition function
        Z = sum([self._unnormalized_likelihood(config) for config in product([0, 1], repeat=n_units)])
        
        if log:
            return sum([np.log(lik) - np.log(Z) for lik in likelihood])
        else:
            return reduce(np.multiply, [lik / Z for lik in likelihood])
    
    def conditional_likelihood(x, cond: dict):
        joint = np.array(x)
        for index, val in cond.items():
            if isinstance(joint[index], int):
                raise
            joint[index] = val

        evidence = [cond.get(i, ...) for i in range(len(x))]

        return self._unnormalized_likelihood(joint) / self.marginal_likelihood(evidence)

# sample from model

In [109]:
def inv_logit(z):
    return 1 / (1 + np.exp(-z))


def gibbs_sampler(weights, biases, init_sample=None, n_samples=100, burn_in=25, every_n=10) -> np.array:
    
    if burn_in > n_samples:
        raise("Can't burn in for more samples than there are in the chain")
        
    init_sample = init_sample or [0 for _ in biases]
    samples = [init_sample]
    
    def _gibbs_step(sample, i):
        z = sum([weights[i, j] * sample[j] for j in range(len(sample)) if j != i]) + biases[i]
        p = inv_logit(z)
        return np.random.binomial(n=1, p=p)
    
    for _ in range(n_samples):
        sample = list(samples[-1])  # make copy
        for i, _ in enumerate(sample):
            sample[i] = _gibbs_step(sample=sample, i=i)
        samples.append( sample )
        
    return np.array([sample for i, sample in enumerate(samples[burn_in:]) if i % every_n == 0])

# update parameters

In [110]:
from itertools import combinations


def update_parameters(weights=weights, biases=biases):
    model_samples = gibbs_sampler(weights=weights, biases=biases, n_samples=1000)

    for i, j in var_combinations:
        # positive phase
        positive_phase = (data[:, i] * data[:, j]).mean()

        # negative phase
        negative_phase = (model_samples[:, i] * model_samples[:, j]).mean()

        # update weights
        weights[i, j] += alpha * (positive_phase - negative_phase)
        
    for i, _ in enumerate(biases):
        # positive phase
        positive_phase = data[:, i].mean()
        
        # negative phase
        negative_phase = model_samples[:, i].mean()
        
        # update biases
        biases[i] += alpha * (positive_phase - negative_phase)
        
    return np.array(weights), np.array(biases)

In [111]:
def plot_n_samples(n, weights=weights, biases=biases):
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    
    samples = gibbs_sampler(n_samples=n, weights=weights, biases=biases)
    x, y, z = zip(*np.array(samples))
    
    x += np.random.randn(len(x)) * .05
    y += np.random.randn(len(y)) * .05
    z += np.random.randn(len(z)) * .05
    
    ax.scatter(x, y, z)

# print(f'sum(x): {sum(x)} | sum(y): {sum(y)} | sum(z): {sum(z)}')

# train model

In [115]:
model = Model(weights=weights, biases=biases)

In [124]:
model.likelihood([[1, 0, 1], [0, 0, 1]])

0.00764502270503839

In [123]:
model.likelihood([0, 0, 1]) * model.likelihood([0, 0, 1])

0.28197655013773254

In [117]:
model.likelihood([0, 0, 1], .)

0.28197655013773254

In [114]:
weights

array([[ 0.26879859,  0.70904794, -1.15882331],
       [-1.03399475, -1.41967822,  0.75022826],
       [-0.16073045,  1.69462416,  0.48787895]])

In [88]:
np.log(model.likelihood([1, 0, 1]))

-1.7770193048567346

In [89]:
model.likelihood([1, 0, 1], log=True)

-1.7770193048567346

In [90]:
model.likelihood([1, 0, 1], log=True) == np.log(model.likelihood([1, 0, 1]))

True

In [69]:
for _ in range(100):
    weights, biases = update_parameters(weights=weights, biases=biases)
    lik = Model(weights=weights, biases=biases).likelihood([1, 0, 1])
    print(lik)
# plot_n_samples(1000)

0.30242740509042776
0.3497552080496797
0.3910308709248392
0.4193612233689859
0.44717897908954407
0.4704027907042031
0.5038645822321107
0.5286349810428999
0.5530611447104644
0.5673064018353872
0.5822006975570396
0.602201617223969
0.618474468048444
0.6390323803927941
0.6561525266696627
0.6649590334317392
0.6741230423303203
0.6839077278979246
0.697914444827443
0.7109174568626483
0.7194558459653888
0.7309047824862285
0.7418897066746011
0.752691432115506
0.7591542723149209
0.7656202770264351
0.7716336381321918
0.7810067502909177
0.7855213912357475
0.7899555120535663
0.797641254206316
0.8026223537118574
0.8075219421916014
0.812032784100748
0.8190318001474552
0.824243094137564
0.8293629288693967
0.8327452116549278
0.834792086529024
0.8364248802693499
0.8387047812549784
0.847413427693415
0.8519419704565414
0.8572649688516429
0.8625088188266651
0.8656968813370706
0.8706557354944301
0.8742804200392055
0.876014213548627
0.8783454031494586
0.8815003465926821
0.883175595194476
0.8855297587056383
0.

In [52]:
model = Model(weights=weights, biases=biases)

In [53]:
model.likelihood([1, 0, 1])

0.9990537166678065

In [21]:
weights

array([[ 0.        , -0.48972103,  3.33757027],
       [-0.48972103,  0.        , -0.60588939],
       [ 3.33757027, -0.60588939,  0.        ]])

In [74]:
model_samples = gibbs_sampler(weights=weights, biases=biases, n_samples=1000)

In [75]:
model_samples.mean(0)

array([1.        , 0.01020408, 0.98979592])

# ask questions

In [10]:
from itertools import product

In [30]:
likelihood(data.ravel())

3.6890563397915714e-06

In [28]:
def H(x):
    return x.T @ weights @ x - biases.T @ x


# def H(x):
#     vals = {
#         (0, 0, 0): -1,
#         (0, 0, 1): -3,
#         (0, 1, 0): -3,
#         (0, 1, 1): 5,
#         (1, 0, 0): 3,
#         (1, 0, 1): -3,
#         (1, 1, 0): -1,
#         (1, 1, 1): 1,
#     }
#     return vals[tuple(x)]


def unnormalized_likelihood(x):
    is_marginal_lik = any([el == ... for el in x])
    if is_marginal_lik:
        unnormalized_lik = 0
        for config in product(*[[0, 1] if el == ... else [el] for el in x]):
            config = np.array(config)
            unnormalized_lik += np.exp(H(config))
    else:
        unnormalized_lik = np.exp(H(x))
    return unnormalized_lik


def likelihood(x):
    """
    Must have the dimensionality of the data observations. To marginalize, put ellipses (...)
    in the elements over which you wish to marginalize.
    """
    numerator = unnormalized_likelihood(x)
    
    denominator = 0
    for config in product([0, 1], repeat=len(x)):
        config = np.array(config)
        denominator += np.exp(H(config))
    return numerator / denominator


def conditional_likelihood(x, cond: dict):
    joint = np.array(x)
    for index, val in cond.items():
        if isinstance(joint[index], int):
            raise
        joint[index] = val
        
    evidence = [cond.get(i, ...) for i in range(len(x))]
    
    return unnormalized_likelihood(joint) / unnormalized_likelihood(evidence)

In [250]:
# P(x_1 = 1 | x_2 = 2) = P(x_1 = 1, x_2 = 2) / P(x_2 = 2)

In [251]:
x = np.array([1, ..., ...])

likelihood(x)

0.13492854264366042

In [252]:
conditional_likelihood(x, cond={2: 0})

0.9799882683585315

In [ ]:
P(x_1 = 1) = P(x_1)

In [46]:
# lets generate data

n_outputs = 10
n_points = 100
data = np.random.binomial(n=1, p=.6, size=(n_points, n_outputs))
weights = np.random.randn(n_points, n_points)
np.fill_diagonal(weights, 0)
bias = np.random.randn(n_outputs)

In [ ]:
# now, derivation of gradient updates